<a href="https://colab.research.google.com/github/ankush-003/NoSQLi-Detector/blob/main/fine_tuning_distilBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 14.7 MB/s eta 0:00:00


In [2]:
from huggingface_hub import notebook_login

notebook_login()

## Loading the dataset

In [3]:
from datasets import load_dataset
dataset = load_dataset("json", data_files="/content/dataset.json")
print(dataset['train'])
print(f"sample data{dataset['train'][0]}")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-920f07ec8741c076/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['text', 'label'],
    num_rows: 10000
})
sample data{'text': '{"username": {"$gt": $}}=', 'label': 1}


## Preprocessing

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [5]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [6]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [11]:
split_dataset = tokenized_dataset["train"].train_test_split(test_size=0.1)

In [12]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [13]:
import evaluate

accuracy = evaluate.load("accuracy")

In [14]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [15]:
id2label = {0: "Benign", 1: "Malitious"}
label2id = {"Benign": 0, "Malitious": 1}

In [16]:
from transformers import create_optimizer
import tensorflow as tf

batch_size = 16
num_epochs = 100
batches_per_epoch = len(split_dataset["train"]) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [17]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [18]:
tf_train_set = model.prepare_tf_dataset(
    split_dataset["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    split_dataset["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


## Training Model

In [19]:
import tensorflow as tf

model.compile(optimizer=optimizer, metrics=["accuracy"])  # No loss argument!

In [20]:
from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)

In [21]:
from transformers.keras_callbacks import PushToHubCallback

push_to_hub_callback = PushToHubCallback(
    output_dir="nosqli_identifier",
    tokenizer=tokenizer,
)

Cloning https://huggingface.co/ankush-003/nosqli_identifier into local empty directory.


Download file tf_model.h5:   0%|          | 1.40k/256M [00:00<?, ?B/s]

Clean file tf_model.h5:   0%|          | 1.00k/256M [00:00<?, ?B/s]

In [22]:
callbacks = [metric_callback, push_to_hub_callback]

In [23]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=3, callbacks=callbacks)

Epoch 1/3
562/562 [==============================] - 2483s 4s/step - loss: 0.0264 - accuracy: 1.0000 - val_loss: 1.9303e-04 - val_accuracy: 1.0000
Epoch 2/3
562/562 [==============================] - 2386s 4s/step - loss: 1.3584e-04 - accuracy: 1.0000 - val_loss: 4.3556e-05 - val_accuracy: 1.0000
Epoch 3/3
562/562 [==============================] - 2415s 4s/step - loss: 4.1736e-05 - accuracy: 1.0000 - val_loss: 1.6974e-05 - val_accuracy: 1.0000


## Prediction

In [24]:
payload = "{\"username\": {\"$gt\": $}}="

In [27]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model="ankush-003/nosqli_identifier")
prediction = classifier(payload)

Some layers from the model checkpoint at ankush-003/nosqli_identifier were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at ankush-003/nosqli_identifier and are newly initialized: ['dropout_79']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
prediction

[{'label': 'Malitious', 'score': 0.9999854564666748}]

## References:
[Text Classification](https://huggingface.co/docs/transformers/tasks/sequence_classification)